In [1]:
print("Installing necessary libraries...")

# Transformers: Lets you easily use powerful pre-trained AI models for tasks like text generation, translation, or classification.
# Accelerate: Helps you run and train models efficiently across CPUs, GPUs.
# BitsAndBytes: Reduces the size of large models using quantization so they can fit and run on limited hardware like free GPUs.
# Torch (PyTorch): A deep learning library used to build, train, and run neural networks using tensors and GPU acceleration.

# Added 'datasets' to the list
!pip install -q transformers accelerate bitsandbytes torch datasets gradio

print("Libraries installed successfully!")

Installing necessary libraries...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.6/68.6 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 28.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 76.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 8.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 31.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 10.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 8.1 MB/s eta 0:00:00:0

In [2]:
# Hugging Face Login

import os
from huggingface_hub import login, notebook_login

print("Attempting Hugging Face login...")

notebook_login()
print("Login successful (or token already present)!")

Attempting Hugging Face login...


Login successful (or token already present)!


In [3]:
import torch
import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset  # Import the dataset loading function
import gradio as gr
from IPython.display import display, Markdown
import random  # To pick random news items

# Check for GPU availability
if torch.cuda.is_available():
    print(f"GPU detected: {torch.cuda.get_device_name(0)}")
    torch.set_default_device("cuda")
    print("PyTorch default device set to CUDA (GPU).")
else:
    print("WARNING: No GPU detected. Performance will be very slow.")
    print("Go to Runtime > Change runtime type and select GPU.")


2025-12-07 05:37:50.719431: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1765085871.133747      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1765085871.255267      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

GPU detected: Tesla T4
PyTorch default device set to CUDA (GPU).


In [4]:
# Helper function for markdown display
def print_markdown(text):
    """Displays text as Markdown."""
    display(Markdown(text))

In [6]:
dataset_id = "mohit9999/all_news_finance_sm_1h2023_custom"

print(f"Loading dataset: {dataset_id}...")

# Load the dataset (will download if not cached)
# We might only need the 'train' split, specify split = 'train' if needed
# The datatype of news_dataset is datasets.Dataset (from the datasets library by Hugging Face).
news_dataset = load_dataset(dataset_id, split = "train")
print("Dataset loaded successfully!")


Loading dataset: mohit9999/all_news_finance_sm_1h2023_custom...


README.md:   0%|          | 0.00/349 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.81M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5062 [00:00<?, ? examples/s]

Dataset loaded successfully!


In [7]:
# Let's Inspect the dataset
news_dataset

Dataset({
    features: ['instruction', 'output', 'text'],
    num_rows: 5062
})

In [8]:
# Let's display the features (columns and their types)
print("\n Dataset Features")
print(news_dataset.features)


 Dataset Features
{'instruction': Value('string'), 'output': Value('string'), 'text': Value('string')}


In [9]:
# Let's view the datasets as a Pandas DataFrame
news_dataset.to_pandas()

,instruction,output,text
0,"Dow drops 400 points, turns negative for the y...",Regional banks led the broader market lower as...,Below is an instruction that describes a task....
1,Bitcoin drops with stocks as analyst warns of ...,Bitcoin dips as the U.S. banking crisis engulf...,Below is an instruction that describes a task....
2,Bitcoin Price Analysis: 29370 Tested After Su...,Bitcoin (BTC/USD) sought to add to recent gain...,Below is an instruction that describes a task....
3,"Bitcoin Is 75% To Halving, Here's How Past Cyc...",The current Bitcoin cycle is now 75% on the wa...,Below is an instruction that describes a task....
4,"Iron Mountain FFO of $0.71 beats by $0.03, rev...",Iron Mountain press release (IRM): Q1 FFO of...,Below is an instruction that describes a task....
...,...,...,...
5057,"Ethereum Merge can trigger high volatility, Bi...",BitMEX CEO Alexander Höptner said that their t...,Below is an instruction that describes a task....
5058,EXPLAINER: Is COVID-19 winding down? Scientist...,Is the coronavirus on its way out? You might t...,Below is an instruction that describes a task....
5059,How Bitcoin Educates The World About Finance,Bitcoin brings financial education to the fore...,Below is an instruction that describes a task....
5060,Crypto Hedge Fund Three Arrows Capital Wallet ...,The removal of liquidity was the first activit...,Below is an instruction that describes a task....


In [32]:
# Let's prepare the data for the LLM
# We'll combine instruction and output for the input text
def combine_news_text(example):

    # Handle potential None values gracefully
    instruction = example.get("instruction", "") or ""
    output = example.get("output", "") or ""

    # Add a separator for clarity
    return {"Full_text": f" instruction: {instruction}\noutput: {output}"}


In [33]:
# Let's apply the function to combine the 'title' and 'description' into 'full_text'
# This uses map, which is efficient for datasets

news_dataset = news_dataset.map(combine_news_text)

print("\n--- Sample Data with 'full_text' ---")
print(news_dataset[0])

Map:   0%|          | 0/5062 [00:00<?, ? examples/s]


--- Sample Data with 'full_text' ---
{'instruction': 'Dow drops 400 points, turns negative for the year as bank fears grow: Live updates', 'output': 'Regional banks led the broader market lower as contagion fears resurfaced.', 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction: Dow drops 400 points, turns negative for the year as bank fears grow: Live updates\n\n### Response: Regional banks led the broader market lower as contagion fears resurfaced.', 'full_text': 'instruction: Dow drops 400 points, turns negative for the year as bank fears grow: Live updates \n output: Regional banks led the broader market lower as contagion fears resurfaced.', 'Full_text': ' instruction: Dow drops 400 points, turns negative for the year as bank fears grow: Live updates\noutput: Regional banks led the broader market lower as contagion fears resurfaced.'}


In [34]:
# Let's display the full_text of the first sample
print(news_dataset[0]["full_text"])

instruction: Dow drops 400 points, turns negative for the year as bank fears grow: Live updates 
 output: Regional banks led the broader market lower as contagion fears resurfaced.


In [35]:
# Let's choose our smaller DeepSeek model, which is suitable for Google Colab and reasoning tasks.
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
print(f"Loading model: {model_id}")
print("This may take a few minutes...")


# AutoModelForCausalLM.from_pretrained(...): Loads a pre-trained language model for tasks like text generation (e.g., ChatGPT-like behavior).
# model_id: The name or path of the model you want to load from Hugging Face (like "meta-llama/Llama-3-8b").
# torch_dtype="auto": Automatically chooses the best data type (like float16 or float32) based on your hardware for efficiency.
# load_in_4bit=True: Loads the model in 4-bit precision to save memory and run on limited hardware like free Colab GPUs.
# device_map="auto": Automatically puts the model on the right device (GPU if available, otherwise CPU).
# trust_remote_code=True: Use only with trusted models to avoid security risks.

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             torch_dtype = "auto",
                                             load_in_4bit = True,
                                             device_map = "auto",
                                             trust_remote_code = True)

model.eval()
print("Model loaded successfully in 4-bit!")

Loading model: deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B
This may take a few minutes...


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Model loaded successfully in 4-bit!


In [36]:
# Let's load the Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code = True)
print("Tokenizer loaded successfully.")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Tokenizer loaded successfully.


In [37]:
# Let's create the Text Generation Pipeline
llm_pipeline = pipeline("text-generation", model = model, tokenizer = tokenizer, torch_dtype = "auto", device_map = "auto")
print("Text generation pipeline created successfully.")

Device set to use cuda:0


Text generation pipeline created successfully.


In [38]:
# Let's build the full formatted prompt:
# <|im_start|>user: Marks the beginning of the user's message.
# {test_question}: Inserts your question.
# <|im_end|>: Marks the end of the user’s message.
# <|im_start|>assistant: Signals that the assistant (AI) is about to reply.
# <think>: Encourages the AI to generate internal reasoning or thoughts before answering.

test_question = "Explain what electric cars are in simple terms. Keep the thinking short."
test_prompt = f"<|im_start|>user\n{test_question}<|im_end|>\n<|im_start|>assistant\n<think>\n"
print(f"Test prompt:\n{test_prompt}")

Test prompt:
<|im_start|>user
Explain what electric cars are in simple terms. Keep the thinking short.<|im_end|>
<|im_start|>assistant
<think>



In [39]:
# Let's test this beast of a model!
# Generate text using the pipeline
# max_new_tokens: Maximum number of tokens to generate
# do_sample: Enable sampling for more diverse outputs (vs greedy decoding). Enables sampling, meaning the model won’t always pick the highest probability next word — instead, it adds some randomness to make responses more creative and diverse.
# temperature: Controls randomness (0.7 is balanced between creative and focused)
# top_p: Nucleus sampling parameter (0.9 keeps responses on topic while allowing variety). Where the model chooses from the top 90% of likely next words (not just the top one). Helps keep output fluent but varied.

print("Testing model with a simple instruction")

outputs = llm_pipeline(test_prompt,
                       max_new_tokens = 4000,
                       do_sample = True,
                       temperature = 0.7,
                       top_p = 0.9)

outputs

Testing model with a simple instruction


[{'generated_text': '<|im_start|>user\nExplain what electric cars are in simple terms. Keep the thinking short.<|im_end|>\n<|im_start|>assistant\n<think>\nOkay, so I\'m trying to explain what electric cars are in simple terms. I know that electric cars use batteries instead of internal combustion engines, but I\'m not entirely sure about the details. Let me break it down step by step.\n\nFirst, I should define electric cars. They are cars that use electricity to power them. That makes sense because they don\'t rely on fuel. I remember hearing terms like "hybrid" cars, which have both electricity and fuel, but electric cars are pure electricity.\n\nNext, I should talk about the main features. They probably have a battery system, which is the heart of how they work. The battery can store a lot of energy, so electric cars can go long distances without refueling. I should mention the battery capacity, maybe using a number like 1,000 to show how much power it holds.\n\nThen, the power sourc

In [40]:
# Function to extract and format the reason and output
def format_model_output(output):

    # Extract the content within <think> tags as Reason
    reason_start = output.find("<think>") + len("<think>")
    reason_end = output.find("</think>")
    reason = output[reason_start:reason_end].strip()

    # Extract the content after </think> as Output
    output_start = reason_end + len("</think>")
    model_output_content = output[output_start:].strip()

    # Format the result
    reason = f"**Reason**:\n{reason}\n"
    output = f"**Output**:\n{model_output_content}"
    return reason, output

In [41]:
# Extract and print the generated text (only the response part)
full_output = outputs[0]["generated_text"]
reason, output = format_model_output(full_output)
print_markdown(reason + output)

**Reason**:
Okay, so I'm trying to explain what electric cars are in simple terms. I know that electric cars use batteries instead of internal combustion engines, but I'm not entirely sure about the details. Let me break it down step by step.

First, I should define electric cars. They are cars that use electricity to power them. That makes sense because they don't rely on fuel. I remember hearing terms like "hybrid" cars, which have both electricity and fuel, but electric cars are pure electricity.

Next, I should talk about the main features. They probably have a battery system, which is the heart of how they work. The battery can store a lot of energy, so electric cars can go long distances without refueling. I should mention the battery capacity, maybe using a number like 1,000 to show how much power it holds.

Then, the power source. Instead of fuel, they use an electric motor or generator to power the car. I should highlight that these motors can work at different speeds, making the car more flexible. Maybe also mention how they can change speed smoothly, which some cars can't do.

Power output is another point. Electric cars can vary in speed and power, which is useful for different driving conditions. Some cars might go from 0 to 60 mph quickly, while others might be more sporty. I should emphasize that electric cars can adjust their power based on the road conditions.

Cost and availability. I should note that electric cars are more expensive than gasoline-powered ones, but they are becoming more affordable. They might be available in different models and sizes, which could affect the driving experience.

I should also mention that electric cars are becoming popular for their efficiency and versatility. They can be used in various environments, from cities to rural areas, which is great for sustainability.

I think I've covered the main points: definition, features, power source, speed and power output, cost, and availability. Let me make sure it's simple and concise, avoiding any technical jargon that might confuse someone new.
**Output**:
Electric cars are cars that use electricity instead of fuel to power them. They have a battery system that stores lots of energy, allowing them to travel long distances without stopping at gas stations. They use electric motors or generators to power the car and can adjust speed and power based on the road conditions. They are becoming popular for their efficiency and versatility, making them great for sustainability in various environments.